In [ ]:
import geopandas as gpd
import json
import shapely
import pandas as pd
import ee
import matplotlib.pyplot as plt


In [ ]:
def get_country_roi(country_name, pop_threshold=0.25, plot=True):
    ee.Initialize()

    dataset = ee.ImageCollection("WorldPop/GP/100m/pop");

    country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', country_name));
    country_bounds = gpd.GeoDataFrame.from_features(country.getInfo()['features'], crs='WGS84')

    data = ee.Image(dataset.select('population').reduce(ee.Reducer.median())).clipToCollection(country);
    zones = data.gt(pop_threshold);
    zones = zones.updateMask(zones.neq(0));

    vectors = zones.addBands(data).reduceToVectors(
        geometry = country,
        crs = data.projection(),
        scale = 2000,
        geometryType = 'polygon',
        eightConnected = True,
        labelProperty = 'zone',
        reducer = ee.Reducer.mean())

    url = vectors.getDownloadURL()
    df = pd.read_csv(url)
    geoms = [shapely.geometry.Polygon(json.loads(g)['coordinates'][0]) for g in df['.geo']]
    polygons = gpd.GeoDataFrame(geometry=geoms, crs='EPSG:4326')

    area = polygons.to_crs('epsg:3857')['geometry'].area.sum() / 10 ** 6
    country_area = (country_bounds.to_crs('epsg:3857')['geometry'].area / 10 ** 6).values[0]
    print(f"{area / country_area:.3%}")
    print(f"Approx DL Tiles: {area / 1000 * 15:.0f}")
    print(f"Approx DL Time: {(area / 1000 * 15) / 500:.2f}")

    if plot:
        fig, ax = plt.subplots(figsize=(8,5), dpi=150)
        country_bounds.plot(ax=ax)
        polygons.plot(ax=ax, color='r')
        plt.xticks([])
        plt.yticks([])
        plt.title(f"{country_name}, Pop Threshold {pop_threshold}\nApprox DL Tiles: {area / 1000 * 15:.0f}\nApprox DL Time: {(area / 1000 * 15) / 500:.2f}")
        plt.show()
    
    polygons.to_file(f'../../data/boundaries/{country_name.lower()}_pop_{pop_threshold}.geojson', driver='GeoJSON')
    return polygons

In [ ]:
polygons = get_country_roi('Indonesia', 0.5)